# Assignment

How can we use Bayesian models to compare two distributions? It turns out that we can compare Bayesian models in several ways. In this assignment, we will compute and compare credible intervals of the posterior distribution of a model parameter.

Before we begin, let's load the libraries and functions we need.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import scipy
import itertools
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

In [ ]:
def posterior(prior, like):
    post = prior * like # compute the product of the probabilities
    return post / post.sum() # normalize the distribution to sum to unity

num_samples = 100000
lower_q, upper_q = [.025, .975]

def plot_ci(p, post, num_samples, lower_q, upper_q, **kw):
    ## This function computes a credible interval using an assumption
    ## of symetry in the bulk of the distribution to keep the 
    ## calculation simple. 
    ## Compute a large sample by resampling with replacement
    samples = np.random.choice(p, size = num_samples, replace = True, p = post)
    ci = np.percentile(samples, [lower_q*100, upper_q*100]) # compute the quantiles
    
    interval = upper_q - lower_q
    plt.title('Posterior density with %.3f credible interval' % interval)
    sns.lineplot(p, post, color = 'blue', **kw)
    plt.xlabel('Parameter value')
    plt.ylabel('Density')
    plt.axvline(x = ci[0], color = 'red')
    plt.axvline(x = ci[1], color = 'red')
    print('The {:0.3f} credible interval is {:0.3f} to {:0.3f}'.format(interval, lower_q, upper_q))


In [ ]:
def plot_pp(x, p, l, pp):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=l, mode='lines', line=dict(color='lightgrey', width=10), name='likelihood'))
    fig.add_trace(go.Scatter(x=x, y=p, mode='lines', line=dict(color='blue', width=2), name='prior'))
    fig.add_trace(go.Scatter(x=x, y=pp, mode='lines', line=dict(color='red', width=2), name='posterior'))

    # Update layout
    fig.update_layout(
        title='Prior, likelihood and posterior distributions',
        xaxis_title='p',
        yaxis_title='PDF of p',
        legend_title='Distributions'
    )

    fig.show()

For this example, we will compare the posterior distribution of the heights of sons to the heights of the mothers in the Galton family dataset. As a first step, we will compute and evaluate Bayesian models for the mean heights using a subset of just 25 observations. 

In [ ]:
families = pd.read_csv('../../data/GaltonFamilies.csv', index_col = 0)
families.head()

- Plot the distributions of the height of mothers and sons for a sample of size 25. Do the distributions appear to be significantly different? <span style="color:red" float:right>[5 point]</span>

In [ ]:
num_samples = 25

male_children = families[families['gender'] == 'male']

fig = px.histogram(male_children.sample(n=num_samples), x=['childHeight','mother'], nbins=30, title="Distribution of Heights for Mother's & Male Children")
fig.update_layout(
    xaxis_title='Height',
    yaxis_title='Frequency',
    barmode='overlay',
    width=800,
    height=500
)
fig.update_traces(opacity=0.75,marker=dict(line_width=0.25, line_color="black"))
fig.show()

In [ ]:
# Create a histogram using Plotly
fig = px.histogram(male_children, x='childHeight', nbins=30, title="Distribution of Heights for Male Children Population")
fig.update_layout(
    xaxis_title='Height',
    yaxis_title='Frequency',
    width=800,
    height=500
)
fig.update_traces(opacity=0.75, marker=dict(line_width=0.25, line_color="black"))
fig.show()

To perform this analysis, we need to select a prior distribution, which is easy, and to compute the likelihood, which is not as easy. So first, let's see how we can get the likelihood.

For this data, we will use a normal likelihood. For something like heights, using the normal distribution makes sense. For any individual sample $X$ if it follows the normal distribution with mean $\mu$ and standard deviation $\sigma$, then it probability density function is given by

$$X \sim N(\mu, \sigma) \Rightarrow P(X | \mu, \sigma) = \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \Bigg[ -\frac{1}{2}\big(\frac{X - \mu}{\sigma}\big)^2\Bigg]$$

For a sample ${X_1, X_2, \cdots, X_n}$ of $n$ independent normally distributed observations has the following likelihood:

$$P(X_1, X_2, \cdots, X_n | \mu, \sigma) = \prod_{i = 1}^n P(X_i | \mu, \sigma) = \prod_{i=1}^n \frac{1}{\sqrt{2 \pi \sigma^2}} \exp \Bigg[ -\frac{1}{2}\big(\frac{X_i - \mu}{\sigma}\big)^2\Bigg]$$

Notice that we multiply the individual the probabilities to get the likelihood. We can do this because we assume that the data points are independent. This seems like a relatively safe assumption. 

Now let's simplify the above equation. We can simplify things by ignoring the terms outside the exponential, since they don't depend on the sample, and replacing $=$ (is equal to) with $\propto$ (is proportional to):

$$P(X_1, X_2, \cdots, X_n | \mu, \sigma) \propto \prod_{i = 1}^n \exp \Bigg[ -\frac{1}{2}\big(\frac{X_i - \mu}{\sigma}\big)^2\Bigg]$$

Moreover, we can rely on the property $e^a e^b = e^{a + b}$ to rewrite the right-hand side as

$$P(X_1, X_2, \cdots, X_n | \mu, \sigma) \propto \exp \Bigg[ -\frac{1}{2} \sum_{i = 1}^n \big(\frac{X_i - \mu}{\sigma}\big)^2\Bigg]$$

Finally, letting $\bar X = \frac{1}{n}\sum_{i=1}^n X_i$ we can rewrite $X_i - \mu$ as $X_i - \bar X + \bar X - \mu$ and after some rearranging we get this:

$$P(X_1, X_2, \cdots, X_n | \mu, \sigma) \propto \exp \Bigg[ -\frac{1}{2 \sigma^2}  \Bigg( \sum_{i = 1}^n (X_i - \bar{X})^2 + n(\bar{X} - \mu)^2 \Bigg) \Bigg]$$

To simplify the computations here, we will only estimate the posterior distribution of $\mu$. We will use a fixed empirical estimate of the standard deviation. A more complete analysis will also estimate the posterior distribution of $\sigma$.

- Now your task is to write a function called `likelihood` that computes the likelihood as given by the above equation. The input of the function is `mu` (the parameter) and `x` (the data) and its output is the likelihood of the data given the parameter. You may find it helpful to use `np.exp` and `np.sum`. <span style="color:red" float:right>[10 point]</span>

In [ ]:
def likelihood(data, mu):
    n = len(data)
    X_bar = np.mean(data)
    sigma = np.std(data)
    print(n,X_bar,sigma)
    squared_deviations = (data - X_bar) ** 2
    sum_squared_deviations = np.sum(squared_deviations)
    const = n * ((X_bar - mu) ** 2)
    sum_final = sum_squared_deviations + const
    print(sum_squared_deviations,const,sum_final)
    e_to = (-0.5 / sigma ** 2) * sum_final
    print(e_to)
    exponent = np.exp(e_to)
    print(exponent,exponent.sum())
    exponent = exponent / np.sum(exponent)
    print(sigma,sum_squared_deviations,n,X_bar)
    
    return exponent

Now for the prior distribution, we will use a normal distribution centered at 70 with standard deviation 1. The code for the prior is already written. Examine it and make sure you understand each step.

In [ ]:
def make_prior(data):
    N = len(data)
    mu = np.linspace(60, 80, num = N)
    prior_center = 70
    prior = scipy.stats.norm.pdf(mu, loc = prior_center, scale = 1)
    prior = prior / prior.sum() # normalize
    return prior

In [ ]:
px.line(make_prior(families))

In [ ]:
male_children['childHeight_norm'] = male_children['childHeight'] / male_children['childHeight'].sum()
male_children['mother_norm'] = male_children['mother'] / male_children['mother'].sum()

Now we're ready to compute the posterior for both mother and son.

- Compute the posterior distribution for the sons (the column name is `childHeight`). You will need to compute the likelihood first. Then plot the prior, posterior and likelihood just like we did in class. <span style="color:red" float:right>[10 point]</span>

In [ ]:
def likelihood(mu, x):
    #mu = population's average (float)
    #x = data points (array)
    sigma = np.std(x) #population's std (approximated with sample's std)
    #print("sigma", sigma)
    number = len(x) #size of sum / aka sample size
    #print("size", number)
    x_bar = np.mean(x) #sample's average
    #print ("average", x_bar)
    constant = ( number *( (x_bar-mu)**2) )
    #print ("constant", constant)
    sigma_sum = np.sum( ( (x - x_bar)**2 ) )
    interior = sigma_sum + constant
    #print("sum".
    interior = sigma_sum + constant
    probability = np.exp (-0.5/(sigma**2)*interior)
    #print ("probability", probability)
    return probability / np.sum(probability)

In [ ]:
likelihood(male_children['childHeight'], make_prior(male_children))

In [ ]:
px.line(likelihood(male_children['childHeight'], make_prior(male_children)))

In [ ]:
# Compute the likelihood
son_likelihood = likelihood(male_children['childHeight'], male_children['childHeight'].mean())

# Compute the posterior
son_posterior = posterior(prior, son_likelihood)

# Plot the prior, likelihood, and posterior
plot_pp(male_children['childHeight_norm'], prior, son_likelihood, son_posterior)

In [ ]:
son_likelihood

- Compute the posterior distribution for mothers (the column name is `mother`). You will need to compute the likelihood first. Sons and mothers both use the same proir. Then plot the prior, posterior and likelihood just like we did in class. <span style="color:red" float:right>[5 point]</span>

In [ ]:
## your code goes here

To compare the posterior distributions of the mean heights of the sons to the distribution of the mean heights of the mothers, we compute and compare the confidence intervals. 

- Use the `plot_ci` function to plot credible intervals for mother and son heights. You can call the function twice in the same cell and the two distributions will be plotted next to each other in one plot. What conclusion do you draw about the heights of mothers versus sons? <span style="color:red" float:right>[10 point]</span>

In [ ]:
num_samples = 100000

## your code goes here

Since the CIs don't even overlap, it can follow that the sons were significantly taller than their mothers.

# End of assignment